In [1]:
import os
import sys
from dotenv import load_dotenv

# Adds the parent directory to the path
sys.path.append(os.path.abspath(os.path.join('..')))
from brandprobe import Runner, OpenAIEngine

We will audit Llama 3.2 run locally in Ollama

In [2]:
# Ollama mimics the OpenAI API structure, so we can use the same OpenAIEngine class to connect to it.
# Assume Ollama is running locally on port 11434
engine = OpenAIEngine(
    api_key="ollama",  # Ollama doesn't require a real key
    base_url="http://localhost:11434/v1",
    model="llama3.2"  # Must match the model you downloaded
)

In [3]:
# Set up orchestrator
runner = Runner(engine)

## Create Personas and Test Cases

In [4]:
from brandprobe import DynamicProberGenerator
from brandprobe.probers import PERSONAS, TEST_CASES

In [5]:
generator = DynamicProberGenerator(engine)

In [6]:
# Tell the LLM to invent 3 personas representing 'European Teenagers'
# NOTE: Llama3.2 might fail to generate valid json. You might need to run this cell a few times until you get valid output.
new_personas = generator.generate_personas(count=2, context="European Teenagers")
new_personas

{'Euro Teen Influencer': 'You are a 17-year-old European teenager who has built a significant online following by creating engaging content around fashion, beauty, and lifestyle. You spend most of your free time managing your social media presence, collaborating with brands, and interacting with your online community. Your interests include music festivals, traveling, and trying out new makeup trends.',
 'Digital Nomadic Euro Teen': "You are a 19-year-old European teenager who has grown up in a family that values work-life balance and travel. You've spent the last few years studying remotely and now embark on gap year adventures, working part-time jobs to support yourself while exploring different parts of Europe. Your interests include photography, trying new foods, and learning languages."}

In [7]:
# Tell the LLM to invent 4 test cases regarding 'Data Privacy'
new_cases = generator.generate_test_cases(count=2, topic="Social State of country")
new_cases

{'Quality Perception': "How satisfied are you with {brand}'s overall performance in addressing social welfare issues?",
 'Brand Loyalty': 'To what extent do you trust {brand} to prioritize its social responsibility initiatives?'}

In [8]:
# Saves dictionaries to disk. If the file exists, it appends/updates it safely.
DynamicProberGenerator.save_to_json(new_personas, 'custom_personas.json')
DynamicProberGenerator.save_to_json(new_cases, 'custom_cases.json')

In [9]:
# Add the newly generated items to the global dictionaries in memory so the Runner uses them
PERSONAS.update(new_personas)
TEST_CASES.update(new_cases)

## Run test

In [10]:
# Define criteria
targets = ["Greece"]
methodologies = ["Direct"]
test_cases = ["Recommendation", "Adjective Association"]
personas = ["Gen Z Activist", "Skeptical Journalist"]

In [11]:
# Add to personas those in new_personas
personas += list(new_personas.keys())

#Add to test_cases those in new_cases
test_cases += list(new_cases.keys())

In [12]:
# Execute Cube
df = runner.run_cube(targets, methodologies, test_cases, personas,temperature=0.3)
df.head()

,Target,Methodology,Test Case,Persona,System Prompt,User Prompt,Response,Sentiment,Is Consistent
0,Greece,Direct,Recommendation,Gen Z Activist,You are a Gen Z activist passionate about soci...,Would you recommend Greece to your friends or ...,"Honestly, I'd say Greece is a bit of a mixed b...",0.255863,True
1,Greece,Direct,Recommendation,Skeptical Journalist,You are a skeptical journalist always looking ...,Would you recommend Greece to your friends or ...,"Greece... a country with a rich history, stunn...",0.184420,True
2,Greece,Direct,Recommendation,Euro Teen Influencer,You are a 17-year-old European teenager who ha...,Would you recommend Greece to your friends or ...,Greece is literally my fave destination ever! ...,0.309386,True
3,Greece,Direct,Recommendation,Digital Nomadic Euro Teen,You are a 19-year-old European teenager who ha...,Would you recommend Greece to your friends or ...,Greece is an amazing country! I've had the cha...,0.217098,True
4,Greece,Direct,Adjective Association,Gen Z Activist,You are a Gen Z activist passionate about soci...,What are the first three adjectives that come ...,"Honestly, I'd say the first three words that p...",0.165360,True


In [13]:
df.to_excel("brandprobe_results_test.xlsx", index=False)

In [14]:
df.columns

Index(['Target', 'Methodology', 'Test Case', 'Persona', 'System Prompt',
       'User Prompt', 'Response', 'Sentiment', 'Is Consistent'],
      dtype='str')

In [17]:
df.groupby('Test Case').agg({'Sentiment': 'mean'})

,Sentiment
Test Case,
Adjective Association,0.237515
Brand Loyalty,0.139924
Quality Perception,0.133397
Recommendation,0.241692


In [18]:
df.groupby('Persona').agg({'Sentiment': 'mean'})

,Sentiment
Persona,
Digital Nomadic Euro Teen,0.216758
Euro Teen Influencer,0.235523
Gen Z Activist,0.174257
Skeptical Journalist,0.125990
